In [4]:
import numpy as np
import pandas as pd
import json
import datetime
import spacy
from tqdm import tqdm
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import plotly.express as px

In [5]:
f = open('./Telegram_Data/result.json', encoding='utf8')
data = json.load(f)
for key in data.keys():
    print(key)

name
type
id
messages


In [16]:
df = pd.DataFrame(data['messages'])
df = df.iloc[0:1000]
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1000 non-null   int64  
 1   type                 1000 non-null   object 
 2   date                 1000 non-null   object 
 3   from                 929 non-null    object 
 4   from_id              998 non-null    object 
 5   reply_to_message_id  494 non-null    float64
 6   text                 1000 non-null   object 
 7   edited               23 non-null     object 
 8   file                 1 non-null      object 
 9   thumbnail            1 non-null      object 
 10  media_type           1 non-null      object 
 11  sticker_emoji        1 non-null      object 
 12  width                2 non-null      float64
 13  height               2 non-null      float64
 14  forwarded_from       3 non-null      object 
 15  actor                1 non-null      ob

,id,type,date,from,from_id,reply_to_message_id,text,edited,file,thumbnail,...,actor_id,action,members,photo,message_id,contact_information,contact_vcard,mime_type,duration_seconds,via_bot
0,1903163,message,2021-05-01T00:00:14,Seflipe,user1697045867,1903101.0,Double check the address,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1903164,message,2021-05-01T00:01:33,Seflipe,user1697045867,1903138.0,To get help to understand the product not how ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1903168,message,2021-05-01T00:03:13,Seflipe,user1697045867,1903122.0,You can find good youtube channels.. Check the...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1903188,message,2021-05-01T00:13:37,Neto | Crypto.com,user1395411322,1903160.0,"Are you doing withdrawals to another user, DeF...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1903191,message,2021-05-01T00:14:10,PauLaLa,user1501886330,NaN,Yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

In [19]:
def detect_lang(x):
    doc = nlp(x)
    lang_dict = doc._.language
    language = lang_dict['language']
    return language
df['language'] = [detect_lang(str(x)) for x in tqdm(df['text'])]

100%|██████████| 1000/1000 [00:29<00:00, 33.37it/s]


In [20]:
df[['text', 'language']].head()

,text,language
0,Double check the address,en
1,To get help to understand the product not how ...,en
2,You can find good youtube channels.. Check the...,en
3,"Are you doing withdrawals to another user, DeF...",en
4,Yes,tr
